In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./db/demande-de-valeurs-foncieres-geolocalisee-bordeaux-metropole.csv", sep=";", encoding="utf-8")

C:\Users\darkj\AppData\Local\Temp\ipykernel_30888\1696475629.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./db/demande-de-valeurs-foncieres-geolocalisee-bordeaux-metropole.csv", sep=";", encoding="utf-8")


In [3]:
print(df.columns)

Index(['Identifiant de mutation (Etalab)', 'Date de la mutation',
       'Numéro de disposition', 'Nature de la mutation', 'Valeur foncière',
       'Numéro de voie', 'Suffixe du numéro (bis/ter)', 'Nom de la voie',
       'Code Fantoir/Rivoli de la voie', 'Code postal',
       'Code INSEE de la commune', 'Nom de la commune',
       'Code INSEE du département', 'Ancien code de la commune',
       'Ancien nom de la commune', 'Identifiant de la parcelle cadastrale',
       'Ancien identifiant de la parcelle cadastrale', 'Numéro du volume',
       'Numéro du lot 1', 'Surface Carrez du lot 1', 'Numéro du lot 2',
       'Surface Carrez du lot 2', 'Numéro du lot 3', 'Surface Carrez du lot 3',
       'Numéro du lot 4', 'Surface Carrez du lot 4', 'Numéro du lot 5',
       'Surface Carrez du lot 5', 'Nombre de lots', 'Code du type de local',
       'Type de local', 'Surface réelle du bâti',
       'Nombre de pièces principales', 'Code de la nature de culture',
       'Nature de culture', 'Code 

In [4]:
df = df.dropna(subset=["Valeur foncière", "Longitude", "Latitude", "Code postal", 'Type de local', 'Surface réelle du bâti'])

In [5]:
df_bordeaux = df[df["Nature de la mutation"] == "Vente"]

In [6]:
df_bordeaux = df[df["Nom de la commune"] == "Bordeaux"]
# print(df_bordeaux)

In [7]:
colonnes_a_garder = ['Date de la mutation', 'Valeur foncière', 'Numéro de voie', 'Suffixe du numéro (bis/ter)', 'Nom de la voie', 'Code postal', 'Nombre de lots', 'Code du type de local',
       'Type de local', 'Surface réelle du bâti', 'Nombre de pièces principales', 'Surface du terrain', 'Longitude', 'Latitude', ]
df_bordeaux = df_bordeaux[colonnes_a_garder]

In [8]:
df_bordeaux.to_csv('./db/Bordeaux.csv', index=False, sep=';')

In [9]:
df_etabl = pd.read_csv("./db/fr-en-annuaire-education.csv", sep=";", encoding="utf-8")

df_etabl_bordeaux = df_etabl[df_etabl['Nom_commune'] == "Bordeaux"]
df_etabl_bordeaux.to_csv('./db/educ_bordeaux.csv', index=False, sep=';')

C:\Users\darkj\AppData\Local\Temp\ipykernel_30888\3352732039.py:1: DtypeWarning: Columns (8,10,18,19,20,39,41,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_etabl = pd.read_csv("./db/fr-en-annuaire-education.csv", sep=";", encoding="utf-8")


In [10]:
df_etabl_bordeaux = pd.read_csv("./db/educ_bordeaux.csv", sep=";", encoding="utf-8")
print(df_etabl_bordeaux.columns)

Index(['Identifiant_de_l_etablissement', 'Nom_etablissement',
       'Type_etablissement', 'Statut_public_prive', 'Adresse_1', 'Adresse_2',
       'Adresse_3', 'Code_postal', 'Code_commune', 'Nom_commune',
       'Code_departement', 'Code_academie', 'Code_region', 'Ecole_maternelle',
       'Ecole_elementaire', 'Voie_generale', 'Voie_technologique',
       'Voie_professionnelle', 'Telephone', 'Fax', 'Web', 'Mail',
       'Restauration', 'Hebergement', 'ULIS', 'Apprentissage', 'Segpa',
       'Section_arts', 'Section_cinema', 'Section_theatre', 'Section_sport',
       'Section_internationale', 'Section_europeenne', 'Lycee_Agricole',
       'Lycee_militaire', 'Lycee_des_metiers', 'Post_BAC',
       'Appartenance_Education_Prioritaire', 'GRETA', 'SIREN_SIRET',
       'Nombre_d_eleves', 'Fiche_onisep', 'position', 'Type_contrat_prive',
       'Libelle_departement', 'Libelle_academie', 'Libelle_region',
       'coordX_origine', 'coordY_origine', 'epsg_origine',
       'nom_circonscription',

In [11]:
df_commerce = pd.read_csv("./db/fi_etabl_p.csv", sep=";", encoding="utf-8")

df_commerce_bordeaux = df_commerce[df_commerce['nom'] == "Bordeaux"]
df_commerce_bordeaux.to_csv('./db/entreprise_bordeaux.csv', index=False, sep=';')

C:\Users\darkj\AppData\Local\Temp\ipykernel_30888\589775896.py:1: DtypeWarning: Columns (5,7,12,19,22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commerce = pd.read_csv("./db/fi_etabl_p.csv", sep=";", encoding="utf-8")


In [12]:
# df_paris = df[df['code_departement'] == 75]
# # print(df_paris)
# df_paris.to_csv('Paris.csv', index=False)

In [13]:
# df_toulouse = df[df['code_departement'] == 31]
# # print(df_toulouse)
# df_toulouse.to_csv('Toulouse.csv', index=False)

In [14]:
# df_nantes = df[df['code_commune'] == 44109]
# # print(df_nantes)
# df_nantes.to_csv('Nantes.csv', index=False)

In [15]:
# df_angouleme = df[df['code_commune'] == 16015]
# # print(df_angouleme)
# df_angouleme.to_csv('Angouleme.csv', index=False)